In [1]:
#This is for asynchronous data responses - to find what data needs to be processed with this script, do as follows:
# go to SP1 Data folder on Outlook >> 4.Asynchronous Survey >> Daily Batched Exports

#output from this script will be an excel file


import csv
import nltk
import enchant
from nltk.corpus import stopwords, words, wordnet # python -m nltk.downloader stopwords
from stop_words import get_stop_words 
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import xlsxwriter
from nltk.stem import PorterStemmer #pip install nltk
from spellchecker import SpellChecker
from colorama import Fore, Style #pip install colorama
from collections import Counter,OrderedDict
import string
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
import re
import unidecode
import pandas as pd

def ReadFile(path, output, unknown):
    Arr=[]
    col_list = ["Q1.4", "Q3.12"]
    df = pd.read_csv(path, usecols=col_list)
    df = df.iloc[2:]
    q = []
    p = []
    df["n"] = df['Q1.4'].str.replace('[^\w\s-]',' ')
    df["c"] = df['Q3.12'].str.replace('[^\w\s-]',' ')
    q = df['n'].to_list()
    p = df['c'].to_list()
    tempp = []
    tempp= [x for x in p if str(x) != 'nan']
    tempq = []
    tempq = [x for x in q if str(x) != 'nan']
    
    tpop =[]
    ppop =[]
    
    punc_list = '.;:!?/\,#@$&)-(\'"'
    replacement =  ' '*len(punc_list)
    for x in tempp:
        new_x = x.translate(str.maketrans(punc_list, replacement))
        tpop.append(new_x)
        
    for x in tempq:
        new_x = x.translate(str.maketrans(punc_list, replacement))
        ppop.append(new_x)
        
        
    flattened = []
    flattened = tpop+ppop
    outputter=[]
    stop_words = list(get_stop_words('en'))         #About 900 stopwords
    nltk_words = list(stopwords.words('english')) #About 150 stopwords
    stop_words.extend(nltk_words) #get array of all stopwords
    flattened = [word for line in flattened for word in line.split()] #consider each word separately
    for w in flattened: #for each word in this array
        if w not in stop_words: #if the word is not a stopword
            outputter.append(w) #put it here
    rep = []
    for x in outputter:
        if len(x) == 1: #chars of 1 length
            outputter.remove(x)
            
    spell = SpellChecker()
    testcheck = []
    x = 'Facebook'
    testcheck += [x]
    x = 'Google'
    testcheck += [x]
    x = 'URM'
    testcheck += [x]
    x = 'LinkedIn'
    testcheck += [x]
    x = 'Reddit'
    testcheck += [x]
    x = 'Twitter'
    testcheck += [x]
    x = 'Bandcamp'
    testcheck += [x]
    
    q = []
    social = []
    for elem in outputter:
        if elem not in testcheck:
            q.append(elem)
        else:
            social.append(elem)
    
    for x in q:
        rep.append(spell.correction(x))
    
    d = enchant.Dict("en_US")
    
    for x in social:
        rep.append(x)
    for x in rep:
        if d.check(x) == False: #unknown words i.e. CU
            unknown.append(x)
        else:
            output.append(x)
def Stems_Frequencies(output, unknown, freq, stem, unknownfreq):
    ps = PorterStemmer()
    tempknown = []
    tempunknown = []
    for x in output: #words stem
        stem.append(ps.stem(x))
    
    for x in stem: #remove all 1 length chars
        if len(x) == 1:
            stem.remove(x)
            
    for x in stem: #gets frequencies of only known words
        freq[x] += 1
    

    tempfreq = Counter()
    for x in unknown:
        x = ps.stem(x)
        tempfreq[x] +=1
        unknownfreq[x] = tempfreq[x]
        
    for x in unknown: #remove duplicates from unknown words list
        if x not in tempunknown:
            tempunknown.append(x)
    unknown.clear()
    
    for x in tempunknown:
        unknown.append(x)
        
    for x in unknown: #unknown words stem
        stem.append(ps.stem(x))
        
    t = len(unknown)
    Newlist =[]

   
    for x in range(len(stem) - t):
        for y in unknown:
            if stem[x] == ps.stem(y):
                Newlist.append(y)
                  
    for x in stem:
        if x not in tempknown: #remove duplicates from stem list
            tempknown.append(x)
    stem.clear()
    
    for x in tempknown:
        
        stem.append(x)    
    
    temps = []
    for x in Newlist:
        if x not in temps:
            temps.append(x)
    li = [i for i in unknown + temps if i not in unknown or i not in temps]
    unknown.clear()
    for x in li:
        unknown.append(x)
    
def Sentiment_Analysis(unknown, output, refinedwords, sentiment, unknownfreq, stem, freq, myfi):
    ps = PorterStemmer()
    tempset = []
    tempwords = []
    for x in output:
        if ps.stem(x) not in tempset and len(ps.stem(x)) !=1 and len((x)) !=1:
            tempset.append(ps.stem(x))
            tempwords.append(x)
    for x in tempwords:
        if x not in refinedwords:
            refinedwords.append(x)
    res = "\n".join("{} {}".format(x, y) for x, y in zip(output, refinedwords))
    for x in refinedwords:
        sid_obj = SentimentIntensityAnalyzer()
        sentiment_dict = sid_obj.polarity_scores(x) 
        if sentiment_dict['compound'] >= 0.05: #if polarity is <0
            y = "Positive"
            st= ps.stem(x)
            sentiment.append(y) #note down that it is a negative word
            #print(x,",",st, ",", y) 
        elif sentiment_dict['compound'] <= -0.05: #if polarity == 0
            y = "Negative"
            st= ps.stem(x)
            sentiment.append(y) #note down that it is a neutral word
            #print(x,",", st, ",", y)
        else: #else
            y = "Neutral"
            st= ps.stem(x)
            sentiment.append(y) #note down that it is a positive word
            #print(x,",", st, ",", y)
    BOLD = '\033[1m'
    END = '\033[0m'
    
    f = open("SavedWords.txt", "r")
    list_of_lists = []
    for line in f:
        stripped_line = line.strip()
        line_list = stripped_line.split()
        list_of_lists.append(line_list)
    f.close()
    f = open("myfi.txt", "a")
    for x in range(len(unknown)):
        unknown[x] = unknown[x].lower()
    cluster = []
    for x in unknown:
        for line in list_of_lists:
            if x in line:
                refinedwords.append(line[0])
                sentiment.append(line[1])
                stem.append(ps.stem(x))
                if x in unknownfreq.keys():
                    y = ps.stem(x)
                    myfi[y] = unknownfreq[x]
                cluster.append(x)
                break;
    leftover = [elem for elem in unknown if elem not in cluster]
    
    f = open("SavedWords.txt", "a")
    for x in leftover:
        newstr = BOLD + x + END
        print("The word ", newstr, " is unrecognized by the English dictionary. Please identify it as Positive, Negative, or Neutral.")
        g = input("Enter your choice: ")
        sentiment.append(g)
        refinedwords.append(x)
        stem.append(ps.stem(x))
        if x in unknownfreq.keys():
            y = ps.stem(x)
            myfi[y] = unknownfreq[x]
        f.write('{0} {1}\n'.format(x, g))
                
    f.close()
def ExportToExcel(sentiment, stem, refinedwords, myfi):
    newfile = input("Enter the file name you'd like to export the results to: ") #get file name
    str2 = '.xlsx'
    filename = newfile+str2
        
    zipped = list(zip(refinedwords, stem, sentiment,myfi.values()))
    p = (sorted(zipped, key = lambda t: t[3], reverse=True))
    
    df = pd.DataFrame(p, columns = ['Word','Stem', 'Sentiment', 'Frequency']) 
    writer = pd.ExcelWriter(filename, engine='xlsxwriter') #create excel doc

    df.to_excel(writer, sheet_name='Sheet1')

    workbook  = writer.book
    worksheet = writer.sheets['Sheet1']

    
    red_format = workbook.add_format({'bg_color':   '#FFC7CE', #formatting for red words
                                  'font_color': '#9C0006'})
    yellow_format = workbook.add_format({'bg_color':   '#FFFF00', #formatting for yellow words
                                  'font_color': '#9C6500'})
    green_format = workbook.add_format({'bg_color': '#C6EFCE', #formatting for green words
                               'font_color': '#006100'})
    orange_format = workbook.add_format({'bg_color': '#FED8B1', #formatting for orange words
                               'font_color': '#FF6600'})
    
    worksheet.conditional_format('D1:D1048576', {'type':     'text', #for all sentiments, make neutral ones yellow
                                       'criteria': 'containing',
                                       'value':    'Neutral',
                                       'format':   yellow_format})
    worksheet.conditional_format('D1:D1048576', {'type':     'text', #for all sentiments, make negative ones red
                                       'criteria': 'containing',
                                       'value':    'Negative',
                                       'format':   red_format})
    worksheet.conditional_format('D1:D1048576', {'type':     'text', #for all sentiments, make positive ones green
                                       'criteria': 'containing',
                                       'value':    'Positive',
                                       'format':   green_format})
    worksheet.conditional_format('D1:D1048576', {'type':     'text', #for all sentiments, make NA orange
                                       'criteria': 'containing',
                                       'value':    'NA',
                                       'format':   orange_format})
    worksheet.conditional_format('D1:D1048576', {'type':     'text', #for all sentiments, make N/A orange
                                       'criteria': 'containing',
                                       'value':    'N/A',
                                       'format':   orange_format})
    writer.save()


    print("--------------")
    print ("A new file by the name of '", filename, "' has been created with your results.")
if __name__ == "__main__": #main menus
    output=[]
    unknown =[]
    unknownfreq = {}
    temp=[]
    refinedwords = []
    stem = [] 
    freq = Counter()
    sentiment = []
    count = Counter() #counter for frequencies
    path1 = input("Enter the name of the file you would like to open: ")
    path2 = '.csv'
    path = path1+path2
    ReadFile(path, output, unknown)
    Stems_Frequencies(output, unknown, freq, stem, unknownfreq)
    myfi = dict(freq)
    Sentiment_Analysis(unknown, output, refinedwords, sentiment, unknownfreq, stem, freq, myfi)
    ExportToExcel(sentiment, stem, refinedwords, myfi)

Enter the name of the file you would like to open: Dec8Data
Enter the file name you'd like to export the results to: Dec8
--------------
A new file by the name of ' Dec8.xlsx ' has been created with your results.
